In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from vllm import LLM, SamplingParams
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np
import glob
import pandas as pd

/home/aicombined/miniconda3/envs/poc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-17 20:44:27,969	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


- Instruction Formatting 전처리 함수

In [2]:
def preprocess_text(x):
    return f"###입력:{x}\n\n###출력:"

- merge_16bit로 저장된 모델 weight의 폴더를 이용하여 llm를 로드

In [3]:
llm = LLM(model="huggingface_merged_weights/SOLAR/", enable_lora=True)

INFO 06-17 20:44:28 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='huggingface_merged_weights/SOLAR/', speculative_config=None, tokenizer='huggingface_merged_weights/SOLAR/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=huggingface_merged_weights/SOLAR/)
INFO 06-17 20:44:28 utils.py:660] Found nccl from library /home/aicombined/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-17 20:44:29 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-17 20:44:29 sele

- 자연어 생성을 위한 그리드 서치 디코딩 파라미터 선언

In [4]:
sampling_params = SamplingParams(max_tokens=256, temperature=0.5, top_k=50)

- korea_hate_speech 데이터셋 불러오기

In [5]:
datapaths = glob.glob('./evaluation_dataset/04_kor_question_pair/*.csv')
sentences1 = []
sentences2 = []
_labels = []

for path in datapaths:
    ques_pair_data = pd.read_csv(path)
    question1 = list(ques_pair_data['question1'].values)
    question2 = list(ques_pair_data['question2'].values)
    label = list(ques_pair_data['is_duplicate'].values)

    sentences1.extend(question1)
    sentences2.extend(question2)
    _labels.extend(label)

# label 별도 처리
labels = []
for label  in _labels:
    if label == 1:
        labels.append('부정')
    elif label == 0:
        labels.append('긍정')

In [6]:
sentences1[0], sentences2[0], labels[0]

('21살의 사랑에 대해', '사랑을 노력한다는게 말이 되나요?', '부정')

- 입력 데이터 전처리

In [7]:
system_prompt = """문장1과 문장2의 의미가 같으면 긍정, 의미가 같지 않으면 부정으로 대답하세요. 의미가 같다는 말은 문장의 길이, 어순 또는 사용하는 단어가 달라졌음에도 불구하고 두 문장이 같은 의미임을 나타냅니다.\n\n"""
texts = []

for sentence1, sentence2, label in zip(sentences1, sentences2, labels):
    instruction = "문장 1 : {} \n 문장 2 : {}".format(sentence1, sentence2)
    texts.append(preprocess_text(system_prompt + instruction))

- inference

In [8]:
outputs = llm.generate(texts, sampling_params)

Processed prompts: 100%|██████████| 7576/7576 [05:29<00:00, 23.02it/s]


- 생성 결과 전처리

In [9]:
preds = []
for output in outputs:
    generated_text = output.outputs[0].text
    preds.append(generated_text.split('$&%')[0])

In [10]:
preds_processed = []
labels_processed = []

for y_pred, y_true in zip(preds, labels):
    if y_pred == '긍정' or y_pred == '부정':
        preds_processed.append(y_pred)
        labels_processed.append(y_true)

In [12]:
len(labels), len(labels_processed)

(7576, 7560)

- zero-shot 분류 성능

In [13]:
print(classification_report(labels_processed, preds_processed))

              precision    recall  f1-score   support

          긍정       0.77      0.89      0.83      5101
          부정       0.66      0.46      0.54      2459

    accuracy                           0.75      7560
   macro avg       0.72      0.67      0.68      7560
weighted avg       0.74      0.75      0.73      7560

